# After years of thought it's finally here

# This is the code that will run at runtime; every elite run of the challenger

The preprocessing code only comes into effect when mons are added to or removed from the database entirely

In [61]:
import pandas as pd
import numpy as np
# import json  # apparently this doesn't need to be imported, pandas handles the pd.read_json

In [62]:
# This dictionary maps attacking types to their coverage
'''
with open(r"jsonfiles/Off.json","r") as p:
    Off=json.load(p)
print(Off)
'''
# Instead of a read/write operation, I just create the dictionary anew 3 cells later.

'\nwith open(r"jsonfiles/Off.json","r") as p:\n    Off=json.load(p)\nprint(Off)\n'

In [63]:
# This dictionary maps move names to their types

# the commented code is obsolete now that I shifted from saving just the dictionary as the json file, to saving the whole dataframe as a json file.
'''
with open(r"jsonfiles/move_dict.json","r") as p:
    move_dict=json.load(p)
print(move_dict)
'''
movetable = pd.read_json(r"jsonfiles/move_dict_table.json") 

In [64]:
movetable

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Acrobatics,Flying,110.0,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Aerial Ace,Flying,60.0,Physical,100,None,NaN,NaN,NaN,Slicing,None,None,NaN
Air Cutter,Flying,60.0,Special,95,None,NaN,NaN,NaN,Slicing,None,None,NaN
Air Slash,Flying,75.0,Special,95,None,NaN,NaN,NaN,Slicing,None,None,NaN
Alluring Voice,Fairy,80.0,Special,100,None,NaN,NaN,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
Wild Charge,Electric,90.0,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Wood Hammer,Grass,120.0,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
X Scissor,Bug,80.0,Physical,100,None,NaN,NaN,NaN,Slicing,None,None,NaN
Zap Cannon,Electric,120.0,Special,50,None,NaN,NaN,NaN,None,None,None,NaN


In [65]:
move_dict=movetable['Type'].to_dict()

In [66]:
# This dataframe has everything you'd need

core_df = pd.read_json(r"jsonfiles/core_df.json")
core_df.tail()

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index,Tier,In Wishlist,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Yanmega,Bug,Flying,None,None,76,116,None,-0.240964,RU,None,...,None,None,None,None,None,None,None,None,None,None
Zangoose,Normal,None,None,None,115,60,None,0.333333,RU,None,...,Payback,Pursuit,Power Trip,Counter,None,None,None,None,None,None
Zarude,Dark,Grass,None,None,120,70,None,0.294118,RU,None,...,None,None,None,None,None,None,None,None,None,None
Zebstrika,Electric,None,None,None,100,80,None,0.133333,RU,None,...,None,None,None,None,None,None,None,None,None,None
Zoroark,Dark,None,None,None,105,120,None,-0.088235,RU,None,...,None,None,None,None,None,None,None,None,None,None


To see the distrubution of the offence index and decide if our chosen margin is alright

In [67]:
core_df.query('-0.24 <= Offense_index <= 0.24').sort_values(by='Offense_index',ascending=True).iloc[:,0:8]
# make sure iloc is the last command. Otherwise the methods after iloc would be operating on a reduced dataset.

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index
Sandy Shocks,Electric,Ground,None,None,81,121,None,-0.233918
Electrode-Hisui,Electric,Grass,None,None,50,80,None,-0.230769
Eldegoss,Grass,None,None,None,50,80,None,-0.230769
Electrode,Electric,None,None,None,50,80,None,-0.230769
Articuno-Galar,Psychic,Flying,None,None,85,125,None,-0.228571
...,...,...,...,...,...,...,...,...
Marowak-Alola,Fire,Ghost,None,None,80,50,None,0.230769
Marowak,Ground,None,None,None,80,50,None,0.230769
Dachsbun,Fairy,None,None,None,80,50,None,0.230769
Dusknoir,Ghost,None,None,None,100,65,None,0.233333


In [68]:
# mons which are considered pure attackers by the above code (just by using ~)
core_df.query('~(-0.24 <= Offense_index <= 0.24)').sort_values(by='Offense_index',ascending=True).iloc[:,0:8]
# make sure iloc is the last command. Otherwise the methods after iloc would be operating on a reduced dataset.

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index
Blissey,Normal,None,None,None,10,75,None,-0.520000
Chandelure,Ghost,Fire,None,None,55,145,None,-0.461538
Nihilego,Rock,Poison,None,None,53,127,None,-0.418079
Togekiss,Fairy,Flying,None,None,50,120,None,-0.411765
Tatsugiri,Dragon,Water,None,None,50,120,None,-0.411765
...,...,...,...,...,...,...,...,...
Avalugg-Hisui,Ice,Rock,None,None,127,34,Strong Jaw,0.525424
Diggersby,Normal,Ground,None,None,161,50,None,0.526066
Sawk,Fighting,None,None,None,125,30,None,0.542857
Darmanitan,Fire,None,None,None,140,30,None,0.578947


at what point is a supereffective move  (with choice specs) from the uninvested mon worth choosing over a basic stab move from the invested stat.

The choice specs is fair even without comparing against a choice band because the surprise, unswitchability, and the wtf factor makes it worth it.

I want something that compares power as well (at least average power). So that high BP moves from the weaker stat will be able to overcome the barrier and get used.


x * 2 * 1.5 >= (y + 50) * 1.5 * 1.1  (the 1.1 is for the + nature). I won't assume that the weaker stat is - nature, because it's easy enough to transfer that to speed or anything.


Let me assume that an average stab move is like ice beam and as 90 power


x * 1.81 >=(y+50)                   # this means, for most mons with the weaker stat above 100, choice specs is viable


now we actually need to replace x and y with not just the stats, but also the move powers.
That's a task for another day.



In [69]:
core_df.loc['Nidoking']

MonType1                  Poison
MonType2                  Ground
MonType3                    None
Freeze_Dry_Stab             None
Atk                          102
SpA                           85
Abilities                   None
Offense_index           0.111842
Tier                          RU
In Wishlist                 None
Used                        None
Trash                       None
Move1                    Outrage
Move2               Dragon Pulse
Move3                Shadow Ball
Move4                Shadow Claw
Move5                        Hex
Move6                Thunderbolt
Move7              Thunder Punch
Move8                  Aqua Tail
Move9                       Surf
Move10               Bubble Beam
Move11              Flamethrower
Move12                Fire Punch
Move13                  Megahorn
Move14                  Ice Beam
Move15                 Ice Punch
Move16                 Avalanche
Move17                 Iron Tail
Move18              Smart Strike
Move19    

In [70]:
#Type Offense dictionary
#mirror coat and counter are typeless

# This has already been loaded as a json file.

Off={           
    'Normal':[],
    'Dragon':['Dragon'],
    'Ghost':['Psychic','Ghost'],
    'Electric':['Water','Flying'],
    'Fairy':['Fighting','Dragon','Dark'],
    'Poison':['Grass','Fairy'],

    'Water':['Fire','Ground','Rock'],
    'Grass':['Water','Ground','Rock'],

    'Fire':['Grass','Ice','Bug','Steel'],
    'Bug':['Grass','Psychic','Dark'],
    
    'Ice':['Grass','Ground','Flying','Dragon'],
    'Freeze Dry':['Water','Grass','Ground','Flying','Dragon'],


    'Psychic':['Fighting','Poison'],
    'Flying':['Grass','Fighting','Bug'],
    'Steel':['Ice','Rock','Fairy'],

    'Rock':['Fire','Ice','Flying','Bug','Dragon'],   #rock is good against nearly half dragons so I added that

    'Fighting':['Normal','Ice','Rock','Dark','Steel'],
    'Ground':['Fire','Electric','Poison','Rock','Steel','Dragon'],    #ground is good against nearly half dragons so I added that
    
    'Dark':['Psychic','Ghost'],
    'Typeless':[],
    'nan' :[]

}

#notes: freeze dry is sneakily coded as an electric type move to include super effectiveness on water

# Let me try out accuracy filtering before making the mon table


In [71]:
movetable.query('Accuracy<90')

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Bone Club,Ground,65.0,Physical,85,None,NaN,NaN,NaN,None,None,None,NaN
Bounce,Flying,85.0,Physical,85,Yes,NaN,NaN,NaN,None,None,None,NaN
Cross Chop,Fighting,100.0,Physical,80,None,NaN,NaN,NaN,None,None,None,NaN
Dragon Rush,Dragon,100.0,Physical,75,None,NaN,NaN,NaN,None,None,None,NaN
Dynamic Punch,Fighting,80.0,Physical,50,None,NaN,NaN,NaN,Punch,None,None,NaN
Focus Blast,Fighting,120.0,Special,70,None,NaN,NaN,NaN,None,None,None,NaN
Gunk Shot,Poison,120.0,Physical,80,None,NaN,NaN,NaN,None,None,None,NaN
Head Smash,Rock,150.0,Physical,80,None,NaN,NaN,NaN,None,None,None,NaN
Hurricane,Flying,110.0,Special,70,None,NaN,NaN,NaN,None,None,None,NaN
Hydro Pump,Water,110.0,Special,80,None,NaN,NaN,NaN,None,None,None,NaN


In [72]:
'''core_df=core_df.replace(to_replace=movetable.query('Accuracy<90').index,value=None)'''
# the issue with this is that I still want gunk shot to be printed even if a mon learns poison jab.
# I want it to filter the mon names based on the criteria, and then access the original filtered database for the mon's moves 
# (It shouldn't access the moves of core_df because that'll print regular neutral moves as well)

"core_df=core_df.replace(to_replace=movetable.query('Accuracy<90').index,value=None)"

In [73]:
#strong_moves_core_df=core_df.copy()

In [74]:
#strong_moves_core_df=strong_moves_core_df.replace(to_replace=movetable.query('Accuracy<90').index,value=None)

In [75]:
#strong_moves_core_df.loc['Lycanroc'] #move 6 is iron tail, and that being none shows that iron tail got filtered out due to being very inaccurate

# doing something similar for base power

In [76]:
import numpy as np

In [77]:
movetable.query('Power.isna()')

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Avalanche,Ice,NaN,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Beat Up,Dark,NaN,None,100,None,NaN,NaN,NaN,None,None,None,NaN
Bone Rush,Ground,NaN,Physical,90,None,NaN,NaN,NaN,None,None,None,NaN
Bullet Seed,Grass,NaN,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Comeuppance,Typeless,NaN,None,100,None,NaN,NaN,NaN,None,None,None,NaN
Counter,Typeless,NaN,None,100,None,NaN,NaN,NaN,None,None,None,NaN
Fling,Dark,NaN,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Grass Knot,Grass,NaN,Special,100,None,NaN,NaN,NaN,None,None,None,NaN
Gyro Ball,Steel,NaN,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Hard Press,Steel,NaN,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN


In [78]:
movetable.query('Power<70 or Power.isna()')

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Aerial Ace,Flying,60.0,Physical,100,None,NaN,NaN,NaN,Slicing,None,None,NaN
Air Cutter,Flying,60.0,Special,95,None,NaN,NaN,NaN,Slicing,None,None,NaN
Aqua Jet,Water,40.0,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Aurora Beam,Ice,65.0,Special,100,None,NaN,NaN,NaN,None,None,None,NaN
Avalanche,Ice,NaN,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
Uturn,Bug,NaN,None,100,None,NaN,NaN,NaN,None,None,None,NaN
Vacuum Wave,Fighting,40.0,Special,100,None,NaN,NaN,NaN,None,None,None,NaN
Venoshock,Poison,NaN,Special,100,None,NaN,NaN,NaN,None,None,None,NaN
Volt Switch,Electric,NaN,None,100,None,NaN,NaN,NaN,None,None,None,NaN


I'm leaving this incomplete for now, I'll just fix it in the next cleaner version of this runtime code

In [79]:
# This code has been copied from the other pokemon file
# doing some core_df with strong_moves_core_df
'''
pre_coverage=core_df.drop(columns=['Atk','SpA','Abilities','Offense_index','Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

'''#move_columns = [col for col in pre_coverage.columns if col.startswith('Move')] #this line of code is only for neatness and to enable the next line of code (since I can't slice a set of columns without loc, and loc involves rows as well)
#pre_coverage[move_columns]=pre_coverage[move_columns].astype(object) # might need this later
'''

coverage=pre_coverage.map(lambda x: Off.get(x))

elite_1='Grass'
elite_2='Fire'
elite_3='Fighting'
elite_4='Ground'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))
finally_filtered =core_df[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]
finally_filtered
'''

"\n\ncoverage=pre_coverage.map(lambda x: Off.get(x))\n\nelite_1='Grass'\nelite_2='Fire'\nelite_3='Fighting'\nelite_4='Ground'\n\n\n\nmask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)\nmask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)\nmask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)\nmask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)\n\nchosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))\nfinally_filtered =core_df[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]\nfinally_filtered\n"

# the massive issue with applying the strong_moves filter here is that
it filters the mons based on all their moves, and removes the mons which need something like bounce to stay super effective on the elite.
This isn't enough, this filter needs to be applied on the physical set of mons, and the special set of mons too.


Otherwise what happens, is that a physical wailord (with its only anti grass move as bounce) shows up in the final excel sheet.
This happens just because its special version learns ice beam for grass mons.
so I have to do the filter on physical and special mons separately

In [80]:
pre_coverage=core_df.drop(columns=['Atk','SpA','Abilities','Offense_index','Tier','In Wishlist','Used','Trash'])
# core_df here used to be strong_moves_core_df, but I moved this part of the code to the phy and special parts.
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

'''#move_columns = [col for col in pre_coverage.columns if col.startswith('Move')] #this line of code is only for neatness and to enable the next line of code (since I can't slice a set of columns without loc, and loc involves rows as well)
#pre_coverage[move_columns]=pre_coverage[move_columns].astype(object) # might need this later
'''

coverage=pre_coverage.map(lambda x: Off.get(x))

elite_1='Normal'
elite_2='Normal'
elite_3='Normal'
elite_4='Normal'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))

# now that we know the chosen mons (which have been chosen even without inaccurate moves), let's use these names to filter out the core_df

In [81]:
chosen_mons.value_counts()

True     330
False    158
Name: count, dtype: int64

In [82]:
pre_coverage=core_df.drop(columns=['Atk','SpA','Abilities','Offense_index','Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

coverage=pre_coverage.map(lambda x: Off.get(x))

#elite_1='Grass'
#elite_2='Ground'
#elite_3='Fighting'
#elite_4='Grass'

mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)
# these new masks give True for inaccurate moves like gunk shot

# we aren't gonna calculate chosen mons now, we need to use the chosen mons that we got from the previous cell


finally_filtered =core_df[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]
finally_filtered

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index,Tier,In Wishlist,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcremie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zebstrika,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### The following code organizes the mons' base typing which is useful against the elite.

In [83]:
# We're processing the base typing separately from the moves because I want 3 columns reserved for the base typing.
# I don't want moves to mesh into the type columns and fill in the null spaces of the type columns.
# so I'm going to concatenate them later

useful_stab=finally_filtered.loc[:,['MonType1','MonType2','MonType3','Freeze_Dry_Stab']] # I want freeeze dry to appear as a stab type, not only when the elite has water, 
# but also when ice is super effective and an elite may bring an ice resist like gastrodon.
# Also even without a water elite, the champ often has a water mon so freeze dry is useful to show.
# This is why I have given freeze dry coverage over 5 types instead of just water.


# if the first type becomes nullified, I"m moving the second type to the spot of the first type. That's what the following code in this cell does.

''' #I'm not trying to force 3 columns anymore
useful_stab = useful_stab.apply(
    lambda row: row.dropna().reset_index(drop=True).reindex(range(3)),   # a bit different from the other code because I'm trying to force 3 columns instead of letting columns shrink too much
    axis=1,
)'''

useful_stab = useful_stab.apply(

    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,

)

useful_stab=useful_stab.rename(
    columns=lambda c: f'SupereffectiveStab{c + 1}'  #scalable no matter whether we've shrunk to 2 columns or 3 columns
)
#useful_stab.columns=['SupereffectiveStab1', 'SupereffectiveStab2', 'SupereffectiveStab3'] # 4 columns might be possible if a 3 column mon gets freeze dry
useful_stab.head(20)

,SupereffectiveStab1
Abomasnow,NaN
Absol,NaN
Accelgor,NaN
Aggron,NaN
Alcremie,NaN
Ambipom,NaN
Ampharos,NaN
Appletun,NaN
Arcanine,NaN
Arcanine-Hisui,NaN


In [84]:
# The stab score is how many montypes are good against the elite (how many montypes remained after the filtering and didn't get nullified.)
# the count method counts the number of non null elements.
useful_stab['StabScore']=useful_stab[[column for column in useful_stab.columns if 'SupereffectiveStab' in column]].count(axis='columns') # If I just pass in useful stab entirely instead of a list of 3 particular columns, then upon rerunning the cell it considers stabscore as a non null column. And that adds 1 to everything
#useful_stab['StabScore'].loc['Aurorus']

In [85]:
# Now that all the freeze dry stuff is done, we should drop that freeze dry column at the end.
# Because otherwise it will be considered in the next block of code
# and the result of this is that this freeze dry column is considered as a move column.
# so aurorus already has the move freeze dry, and it also has a fake move column which has the value freeze dry.
# this is why the freeze dry column should be dropped, so that it doesn't act like a fake move.

# okay now that I've moved this freeze dry column to the stab section, it won't interfere with the move columns. (Of the next code)
# so all the above comments are useless.


#finally_filtered=finally_filtered.drop(columns='Freeze_Dry_Stab')
# now this column automatically gets dropped because useful_stab has reshaped itself.

## The following code is to neaten the dataframe: to remove null values and slide all the relevant data left. Maybe it can be converted into a function.

This code also uses the usefu_stab group of columns and concatenates them with the random information columns and the move columns.
So useful stab 

In [86]:
df_where_NaNs_are_removable=finally_filtered.loc[:,'Move1':] # this dataframe ignores the stab type, and has the Moves.

df_where_NaNs_are_removable = df_where_NaNs_are_removable.apply(
    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,
)

df_where_NaNs_are_removable = df_where_NaNs_are_removable.rename(
    columns=lambda c: f'Move{c + 1}'
)
df_where_NaNs_are_removable

,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10
Abomasnow,Brick Break,Low Kick,Body Press,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Close Combat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Focus Blast,Drain Punch,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Body Press,Low Kick,Superpower,Focus Blast,Dynamic Punch,Reversal,NaN,NaN,NaN,NaN
Alcremie,Drain Punch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Yanmega,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Reversal,Close Combat,Focus Blast,Low Kick,Dynamic Punch,Aura Sphere,Brick Break,Low Sweep,Revenge,NaN
Zarude,Close Combat,Low Kick,Hammer Arm,Low Sweep,Revenge,NaN,NaN,NaN,NaN,NaN
Zebstrika,Low Kick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


have a filter to avoid inconsistent moves like fling, pursuit
maybe bulldoze and rock tomb too, although those moves could come under another filter

In [87]:
#final=pd.concat([finally_filtered.loc[:,:'Trash'],useful_stab,df_where_NaNs_are_removable], axis=1)

#everything_before_move1=finally_filtered.loc[:,:finally_filtered.columns[finally_filtered.columns.get_loc('Move1')-1]]
# I just realized that for this simple thing I could have just done iloc. But it appears iloc shouldn't have the -1.
everything_before_move1=finally_filtered.iloc[:,:finally_filtered.columns.get_loc('Move1')]

# finding the index of Move1, and subtracting 1 gives us the previous column index. inputting that into finally_filtered.columns gives us the column name.
# refer the tosplit, part_1, part_2 stuff around SrijanSort (from preprocessing) to understand the above line, and why I stopped using 'Trash' as the endpoint of slicing.

In [88]:
# I still need to add all those wishlist, used, tier values back
everything_before_move1[['Tier','In Wishlist','Used','Trash']]=core_df[['Tier','In Wishlist','Used','Trash']]
# finally_filtered (and everything_before_move_1) did have these 4 column names (in the right spots too), but they were initially null (due to .map() stuff long ago). 
# With this code, I set all the values to the values found in core_df.

In [89]:
# I've realized I also want to be able to see the full typing of the mons even if the full typing isn't useful offensively
# So at one side of the table I'll have the full typing, and at the other side I'll have the super effective typing
# so I'm directly copying the original typing too

everything_before_move1[['MonType1','MonType2','MonType3']]=core_df[['MonType1','MonType2','MonType3']]

# The Freeze dry column doesn't get added here because it's unnecessary. I don't want to look at it everytime alongside ice when both are non supereffective.

I'm kinda cooked when it comes to megas. When the megas are also RU, for now, I'll give the mons the stats of their megas instead of their regular stats.
I'll also lump all abilities together, mega or not.

In [90]:
everything_before_move1[['Atk','SpA','Abilities','Offense_index']]=core_df[['Atk','SpA','Abilities','Offense_index']]

Function to see whether a mon's lower stat is even viable or not (whether it's primarily physical or special will be coded later)

In [91]:
final=pd.concat([everything_before_move1,useful_stab,df_where_NaNs_are_removable], axis=1)
final

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index,Tier,In Wishlist,...,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10
Abomasnow,Grass,Ice,None,NaN,132,132,None,0.000000,RU,None,...,Brick Break,Low Kick,Body Press,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,None,None,NaN,150,115,None,0.175000,RU,None,...,Close Combat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,None,None,NaN,70,100,None,-0.200000,RU,None,...,Focus Blast,Drain Punch,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,None,NaN,140,60,None,0.421053,RU,None,...,Body Press,Low Kick,Superpower,Focus Blast,Dynamic Punch,Reversal,NaN,NaN,NaN,NaN
Alcremie,Fairy,None,None,NaN,60,110,None,-0.312500,RU,None,...,Drain Punch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,None,NaN,76,116,None,-0.240964,RU,None,...,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,None,None,NaN,115,60,None,0.333333,RU,None,...,Reversal,Close Combat,Focus Blast,Low Kick,Dynamic Punch,Aura Sphere,Brick Break,Low Sweep,Revenge,NaN
Zarude,Dark,Grass,None,NaN,120,70,None,0.294118,RU,None,...,Close Combat,Low Kick,Hammer Arm,Low Sweep,Revenge,NaN,NaN,NaN,NaN,NaN
Zebstrika,Electric,None,None,NaN,100,80,None,0.133333,RU,None,...,Low Kick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Code to make it sortable by stab

In [92]:
final=final.sort_values(by='StabScore', kind = 'stable', ascending=False) # The kind parameter is needed to keep the sorting stable.

In [93]:
final=final.drop(columns='StabScore') # Now that it's sorted, we no longer need this StabScore

In [94]:
# A few stuff I don't need for now, might change later
final=final.drop(columns=['Tier','In Wishlist', 'Trash'])

In [95]:
final.to_csv('Best_Mons_at_runtime_Versus_The_Elite.csv')

copy paste technician mons before and after all the move deletions so that they don't lose their moves
Also Strong Jaw mons and stuff ig

# Now a bit where I find all mons that can be good without special moves. I find all the mons that can be good without physical moves. Then I combine them.
Then I pass them through the elite 4 masks all over again, neatify all over again, and export them (In a future code I'll turn these processes into functions)

In [96]:
movetable

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Acrobatics,Flying,110.0,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Aerial Ace,Flying,60.0,Physical,100,None,NaN,NaN,NaN,Slicing,None,None,NaN
Air Cutter,Flying,60.0,Special,95,None,NaN,NaN,NaN,Slicing,None,None,NaN
Air Slash,Flying,75.0,Special,95,None,NaN,NaN,NaN,Slicing,None,None,NaN
Alluring Voice,Fairy,80.0,Special,100,None,NaN,NaN,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
Wild Charge,Electric,90.0,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
Wood Hammer,Grass,120.0,Physical,100,None,NaN,NaN,NaN,None,None,None,NaN
X Scissor,Bug,80.0,Physical,100,None,NaN,NaN,NaN,Slicing,None,None,NaN
Zap Cannon,Electric,120.0,Special,50,None,NaN,NaN,NaN,None,None,None,NaN


In [97]:
physical_moves=movetable.query('Category == "Physical"').index.to_list()
physical_moves

['Acrobatics',
 'Aerial Ace',
 'Aqua Cutter',
 'Aqua Jet',
 'Aqua Tail',
 'Avalanche',
 'Axe Kick',
 'Bite',
 'Blaze Kick',
 'Bone Club',
 'Bone Rush',
 'Bonemerang',
 'Bounce',
 'Brave Bird',
 'Breaking Swipe',
 'Brick Break',
 'Brutal Swing',
 'Bulldoze',
 'Bullet Punch',
 'Bullet Seed',
 'Close Combat',
 'Crabhammer',
 'Cross Chop',
 'Cross Poison',
 'Crunch',
 'Darkest Lairat',
 'Dig',
 'Dive',
 'Dragon Claw',
 'Dragon Hammer',
 'Dragon Rush',
 'Drain Punch',
 'Drill Peck',
 'Drill Run',
 'Dual Chop',
 'Dual Wingbeat',
 'Dynamic Punch',
 'Earthquake',
 'Fire Fang',
 'Fire Punch',
 'First Impression',
 'Flare Blitz',
 'Fling',
 'Flip Turn',
 'Fly',
 'Fury Cutter',
 'Gunk Shot',
 'Gyro Ball',
 'Hammer Arm',
 'Hard Press',
 'Head Smash',
 'Headlong Rush',
 'Heat Crash',
 'Heavy Slam',
 'High Horsepower',
 'High Jump Kick',
 'Ice Fang',
 'Ice Hammer',
 'Ice Punch',
 'Ice Shard',
 'Ice Spinner',
 'Icicle Spear',
 'Iron Head',
 'Iron Tail',
 'Jaw Lock',
 'Jump Kick',
 'Knock Off',
 'Lash

In [98]:
special_moves=movetable.query('Category == "Special"').index.to_list()
special_moves

['Air Cutter',
 'Air Slash',
 'Alluring Voice',
 'Aura Sphere',
 'Aurora Beam',
 'Belch',
 'Boomburst',
 'Brine',
 'Bubble Beam',
 'Bug Buzz',
 'Burning Jealousy',
 'Chilling Water',
 'Dark Pulse',
 'Dazzling Gleam',
 'Discharge',
 'Draco Meteor',
 'Dragon Pulse',
 'Draining Kiss',
 'Earth Power',
 'Energy Ball',
 'Expanding Force',
 'Extrasensory',
 'Flame Burst',
 'Flamethrower',
 'Flash Cannon',
 'Focus Blast',
 'Freeze Dry',
 'Future Sight',
 'Giga Drain',
 'Grass Knot',
 'Heat Wave',
 'Hex',
 'Hurricane',
 'Hydro Pump',
 'Ice Beam',
 'Inferno',
 'Meteor Beam',
 'Mirror Shot',
 'Misty Explosion',
 'Moonblast',
 'Mud Bomb',
 'Muddy Water',
 'Mystical Fire',
 'Octazooka',
 'Overheat',
 'Petal Dance',
 'Pollen Puff',
 'Power Gem',
 'Psybeam',
 'Psychic',
 'Psychic Noise',
 'Psyshock',
 'Scald',
 'Scorching Sands',
 'Shadow Ball',
 'Shock Wave',
 'Signal Beam',
 'Sludge',
 'Sludge Bomb',
 'Sludge Wave',
 'Snarl',
 'Solar Beam',
 'Stored Power',
 'Surf',
 'Synchronoise',
 'Thunder',
 'T

In [99]:
# this dataframe is basically purely the moves part
df_where_NaNs_are_removable

,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10
Abomasnow,Brick Break,Low Kick,Body Press,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Close Combat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Focus Blast,Drain Punch,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Body Press,Low Kick,Superpower,Focus Blast,Dynamic Punch,Reversal,NaN,NaN,NaN,NaN
Alcremie,Drain Punch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Yanmega,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Reversal,Close Combat,Focus Blast,Low Kick,Dynamic Punch,Aura Sphere,Brick Break,Low Sweep,Revenge,NaN
Zarude,Close Combat,Low Kick,Hammer Arm,Low Sweep,Revenge,NaN,NaN,NaN,NaN,NaN
Zebstrika,Low Kick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
physical_gone = df_where_NaNs_are_removable.replace(to_replace=physical_moves,value=None)

In [101]:
special_gone = df_where_NaNs_are_removable.replace(to_replace=special_moves,value=None)

In [102]:
#clean_df=clean_df.replace(to_replace=["Fling","Pursuit","Bulldoze","Rock Tomb","Dig"],value=None)

# this was the cell I used to make certain cells null

# Now to continue the regular process.
First neatify these dataframes. 


In [103]:
physical_gone = physical_gone.apply(
    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,
)

physical_gone = physical_gone.rename(
    columns=lambda c: f'Move{c + 1}'
)
physical_gone

,Move1,Move2,Move3
Abomasnow,Body Press,NaN,NaN
Absol,NaN,NaN,NaN
Accelgor,Focus Blast,NaN,NaN
Aggron,Body Press,Focus Blast,NaN
Alcremie,NaN,NaN,NaN
...,...,...,...
Yanmega,NaN,NaN,NaN
Zangoose,Focus Blast,Aura Sphere,NaN
Zarude,NaN,NaN,NaN
Zebstrika,NaN,NaN,NaN


In [104]:
special_gone = special_gone.apply(
    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,
)

special_gone = special_gone.rename(
    columns=lambda c: f'Move{c + 1}'
)
special_gone

,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9
Abomasnow,Brick Break,Low Kick,Body Press,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Close Combat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Drain Punch,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Body Press,Low Kick,Superpower,Dynamic Punch,Reversal,NaN,NaN,NaN,NaN
Alcremie,Drain Punch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
Yanmega,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Reversal,Close Combat,Low Kick,Dynamic Punch,Brick Break,Low Sweep,Revenge,NaN,NaN
Zarude,Close Combat,Low Kick,Hammer Arm,Low Sweep,Revenge,NaN,NaN,NaN,NaN
Zebstrika,Low Kick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
full_df_special=pd.concat([everything_before_move1,physical_gone], axis=1)
full_df_physical=pd.concat([everything_before_move1,special_gone], axis=1)

In [106]:
#full_df_special=full_df_special.drop(columns='StabScore')
#full_df_physical=full_df_physical.drop(columns='StabScore')

# time to put everything through the wringer again
(to check if these robbed mons are still usable for the elite)

In [107]:
strong_moves_special=full_df_special.copy()

In [108]:
strong_moves_special=strong_moves_special.replace(to_replace=movetable.query('Accuracy<90 or Power<70').index,value=None)

In [109]:
# This code has been copied from the other pokemon file

pre_coverage=strong_moves_special.drop(columns=['Atk','SpA','Abilities','Offense_index','Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

'''move_columns = [col for col in pre_coverage.columns if col.startswith('Move')] #this line of code is only for neatness and to enable the next line of code (since I can't slice a set of columns without loc, and loc involves rows as well)
pre_coverage[move_columns]=pre_coverage[move_columns].astype(object)''' # might need this later

coverage=pre_coverage.map(lambda x: Off.get(x))

#elite_1='Grass'
#elite_2='Ground'
#elite_3='Fighting'
#elite_4='Grass'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))

In [110]:
pre_coverage=full_df_special.drop(columns=['Atk','SpA','Abilities','Offense_index','Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

'''move_columns = [col for col in pre_coverage.columns if col.startswith('Move')] #this line of code is only for neatness and to enable the next line of code (since I can't slice a set of columns without loc, and loc involves rows as well)
pre_coverage[move_columns]=pre_coverage[move_columns].astype(object)''' # might need this later

coverage=pre_coverage.map(lambda x: Off.get(x))

#elite_1='Grass'
#elite_2='Ground'
#elite_3='Fighting'
#elite_4='Grass'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

#chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))
finally_filtered =full_df_special[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]


useful_stab=finally_filtered.loc[:,['MonType1','MonType2','MonType3','Freeze_Dry_Stab']]
useful_stab = useful_stab.apply(

    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,

)

useful_stab=useful_stab.rename(
    columns=lambda c: f'SupereffectiveStab{c + 1}'  #scalable no matter whether we've shrunk to 2 columns or 3 columns
)
useful_stab['StabScore']=useful_stab[[column for column in useful_stab.columns if 'SupereffectiveStab' in column]].count(axis='columns')

df_where_NaNs_are_removable=finally_filtered.loc[:,'Move1':] # this dataframe ignores the stab type, and has the Moves.

df_where_NaNs_are_removable = df_where_NaNs_are_removable.apply(
    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,
)

df_where_NaNs_are_removable = df_where_NaNs_are_removable.rename(
    columns=lambda c: f'Move{c + 1}'
)
df_where_NaNs_are_removable
everything_before_move1=finally_filtered.iloc[:,:finally_filtered.columns.get_loc('Move1')]
everything_before_move1[['Tier','In Wishlist','Used','Trash','MonType1','MonType2','MonType3','Atk','SpA','Abilities','Offense_index']]=full_df_special[['Tier','In Wishlist','Used','Trash','MonType1','MonType2','MonType3','Atk','SpA','Abilities','Offense_index']]

final=pd.concat([everything_before_move1,useful_stab,df_where_NaNs_are_removable], axis=1)
final
final=final.sort_values(by='StabScore', kind = 'stable', ascending=False)
#final=final.drop(columns='StabScore')
special_final=final.drop(columns=['Tier','In Wishlist', 'Trash'])




special_final#.to_csv('Best_Mons_at_runtime_Versus_The_Elite_special.csv')



special_final = special_final.rename(
    index=lambda c: f'{c}_special'
)
#special_final.info()
special_final=special_final.query('Offense_index <= 0.24') # if the physical attack is higher than the special attack by a factor of 0.24, we really don't wanna use special attack, which is bound to be trash
special_final

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index,Used,SupereffectiveStab1,StabScore,Move1,Move2,Move3
Chesnaught_special,Grass,Fighting,None,NaN,107,74,None,0.210191,None,Fighting,1,NaN,NaN,NaN
Cobalion_special,Steel,Fighting,None,NaN,90,90,None,0.000000,Yes,Fighting,1,NaN,NaN,NaN
Decidueye-Hisui_special,Grass,Fighting,None,NaN,112,95,None,0.104938,Yes,Fighting,1,NaN,NaN,NaN
Emboar_special,Fire,Fighting,None,NaN,123,100,None,0.132948,None,Fighting,1,NaN,NaN,NaN
Falinks_special,Fighting,None,None,NaN,100,70,None,0.200000,None,Fighting,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Volcanion_special,Fire,Water,None,NaN,110,130,None,-0.111111,None,NaN,0,Focus Blast,Body Press,NaN
Wailord_special,Water,None,None,NaN,90,90,None,0.000000,None,NaN,0,Body Press,NaN,NaN
Walrein_special,Ice,Water,None,NaN,80,95,None,-0.103448,None,NaN,0,Body Press,NaN,NaN
Wigglytuff_special,Normal,Fairy,None,NaN,70,85,None,-0.111111,None,NaN,0,Body Press,NaN,NaN


In [111]:
strong_moves_physical=full_df_physical.copy()

In [112]:
strong_moves_physical=strong_moves_physical.replace(to_replace=movetable.query('Accuracy<90 or Power<70').index,value=None)

In [113]:
# This code has been copied from the other pokemon file

pre_coverage=strong_moves_physical.drop(columns=['Atk','SpA','Abilities','Offense_index','Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

'''move_columns = [col for col in pre_coverage.columns if col.startswith('Move')] #this line of code is only for neatness and to enable the next line of code (since I can't slice a set of columns without loc, and loc involves rows as well)
pre_coverage[move_columns]=pre_coverage[move_columns].astype(object)''' # might need this later

coverage=pre_coverage.map(lambda x: Off.get(x))

#elite_1='Grass'
#elite_2='Ground'
#elite_3='Fighting'
#elite_4='Grass'


mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))

In [114]:
# This code has been copied from the other pokemon file

pre_coverage=full_df_physical.drop(columns=['Atk','SpA','Abilities','Offense_index','Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

'''move_columns = [col for col in pre_coverage.columns if col.startswith('Move')] #this line of code is only for neatness and to enable the next line of code (since I can't slice a set of columns without loc, and loc involves rows as well)
pre_coverage[move_columns]=pre_coverage[move_columns].astype(object)''' # might need this later

coverage=pre_coverage.map(lambda x: Off.get(x))

#elite_1='Grass'
#elite_2='Ground'
#elite_3='Fighting'
#elite_4='Grass'


mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

#chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))
finally_filtered =full_df_physical[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]


useful_stab=finally_filtered.loc[:,['MonType1','MonType2','MonType3','Freeze_Dry_Stab']]
useful_stab = useful_stab.apply(

    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,

)

useful_stab=useful_stab.rename(
    columns=lambda c: f'SupereffectiveStab{c + 1}'  #scalable no matter whether we've shrunk to 2 columns or 3 columns
)
useful_stab['StabScore']=useful_stab[[column for column in useful_stab.columns if 'SupereffectiveStab' in column]].count(axis='columns')

df_where_NaNs_are_removable=finally_filtered.loc[:,'Move1':] # this dataframe ignores the stab type, and has the Moves.

df_where_NaNs_are_removable = df_where_NaNs_are_removable.apply(
    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,
)

df_where_NaNs_are_removable = df_where_NaNs_are_removable.rename(
    columns=lambda c: f'Move{c + 1}'
)
df_where_NaNs_are_removable
everything_before_move1=finally_filtered.iloc[:,:finally_filtered.columns.get_loc('Move1')]
everything_before_move1[['Tier','In Wishlist','Used','Trash','MonType1','MonType2','MonType3','Atk','SpA','Abilities','Offense_index']]=full_df_physical[['Tier','In Wishlist','Used','Trash','MonType1','MonType2','MonType3','Atk','SpA','Abilities','Offense_index']]

final=pd.concat([everything_before_move1,useful_stab,df_where_NaNs_are_removable], axis=1)
final
final=final.sort_values(by='StabScore', kind = 'stable', ascending=False)
#final=final.drop(columns='StabScore')
physical_final=final.drop(columns=['Tier','In Wishlist', 'Trash'])
physical_final#.to_csv('Best_Mons_at_runtime_Versus_The_Elite_physical.csv')




physical_final = physical_final.rename(
    index=lambda c: f'{c}_physical'
)
physical_final=physical_final.query('-0.24 <= Offense_index') # if the special attack is higher than the physical attack by a factor of 0.24, we really don't wanna use physical attack, which is bound to be trash
physical_final

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index,Used,SupereffectiveStab1,StabScore,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9
Bewear_physical,Normal,Fighting,None,NaN,125,55,None,0.400000,None,Fighting,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Breloom_physical,Grass,Fighting,None,NaN,130,60,None,0.388889,None,Fighting,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chesnaught_physical,Grass,Fighting,None,NaN,107,74,None,0.210191,None,Fighting,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cobalion_physical,Steel,Fighting,None,NaN,90,90,None,0.000000,Yes,Fighting,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Crabominable_physical,Fighting,Ice,None,NaN,132,62,None,0.384615,None,Fighting,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wyrdeer_physical,Normal,Psychic,None,NaN,105,105,None,0.000000,Yes,NaN,0,Jump Kick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose_physical,Normal,None,None,NaN,115,60,None,0.333333,None,NaN,0,Reversal,Close Combat,Low Kick,Dynamic Punch,Brick Break,Low Sweep,Revenge,NaN,NaN
Zarude_physical,Dark,Grass,None,NaN,120,70,None,0.294118,None,NaN,0,Close Combat,Low Kick,Hammer Arm,Low Sweep,Revenge,NaN,NaN,NaN,NaN
Zebstrika_physical,Electric,None,None,NaN,100,80,None,0.133333,None,NaN,0,Low Kick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
phy_spe_split=pd.concat([physical_final,special_final],axis=0)
#phy_spe_split.loc['Delibird_physical':,"Used":]
# this delibird code to check things obviously fails when delibird isn't in the set

In [116]:
phy_spe_split.columns # this appears not to have trash, wishlist, tier columns anymore (I'll fix that eventually ig)

Index(['MonType1', 'MonType2', 'MonType3', 'Freeze_Dry_Stab', 'Atk', 'SpA',
       'Abilities', 'Offense_index', 'Used', 'SupereffectiveStab1',
       'StabScore', 'Move1', 'Move2', 'Move3', 'Move4', 'Move5', 'Move6',
       'Move7', 'Move8', 'Move9'],
      dtype='object')

In [117]:
phy_spe_split=phy_spe_split.sort_index().sort_values(by='StabScore', kind = 'stable', ascending=False)

##### In the future, if a physical mon learns only one relevant special move (like power gem meowscarada vs mono fire),
I might still want to keep that and then display that special move in Meowscarada_spe right below meowscarada physical (IN THE LOWER BRACKET OF MONS WHICH ARE DEPENDENT ON ONE OPPOSITE CATEGORY MOVE) (the next tables that are printed after the most most desirable mons)

In [118]:
phy_spe_split=phy_spe_split.drop(columns='StabScore')

In [119]:
phy_spe_split.to_csv(f'{elite_1}_{elite_2}_{elite_3}_{elite_4}.csv')

The code is unoptimized, I just messed around with columns (In the future I should be able to drop and add the relevant columns)


I think I'll probably just do that in the next code file. 
I plan to make this code file obsolete.